### Vector Store and Retriever
https://reference.langchain.com/python/langchain-core/documents/base/Document

In [4]:
from langchain_core.documents import Document

documents = [
    Document(
        page_content="Node.js is widely used for building scalable backend services.",
        metadata={"source": "https://nodejs.org"},
    ),
    Document(
        page_content="MongoDB aggregation pipelines allow complex data transformations.",
        metadata={"source": "https://mongodb.com/docs"},
    ),
    Document(
        page_content="JavaScript supports asynchronous programming using promises and async/await.",
        metadata={"source": "https://developer.mozilla.org"},
    ),
    Document(
        page_content="Express.js simplifies API development in Node.js applications.",
        metadata={"source": "https://expressjs.com"},
    ),
    Document(
        page_content="Redis is often used for caching to improve application performance.",
        metadata={"source": "https://redis.io"},
    ),
    Document(
        page_content="JWT tokens are commonly used for stateless authentication.",
        metadata={"source": "https://jwt.io"},
    ),
    Document(
        page_content="Docker containers help maintain consistent development environments.",
        metadata={"source": "https://docker.com"},
    ),
    Document(
        page_content="PostgreSQL is a powerful open-source relational database system.",
        metadata={"source": "https://postgresql.org"},
    ),
    Document(
        page_content="REST APIs follow standard HTTP methods like GET, POST, PUT, DELETE.",
        metadata={"source": "https://restfulapi.net"},
    ),
    Document(
        page_content="TypeScript adds static typing capabilities on top of JavaScript.",
        metadata={"source": "https://typescriptlang.org"},
    ),
]
documents

[Document(metadata={'source': 'https://nodejs.org'}, page_content='Node.js is widely used for building scalable backend services.'),
 Document(metadata={'source': 'https://mongodb.com/docs'}, page_content='MongoDB aggregation pipelines allow complex data transformations.'),
 Document(metadata={'source': 'https://developer.mozilla.org'}, page_content='JavaScript supports asynchronous programming using promises and async/await.'),
 Document(metadata={'source': 'https://expressjs.com'}, page_content='Express.js simplifies API development in Node.js applications.'),
 Document(metadata={'source': 'https://redis.io'}, page_content='Redis is often used for caching to improve application performance.'),
 Document(metadata={'source': 'https://jwt.io'}, page_content='JWT tokens are commonly used for stateless authentication.'),
 Document(metadata={'source': 'https://docker.com'}, page_content='Docker containers help maintain consistent development environments.'),
 Document(metadata={'source': '

In [5]:
import os
from dotenv import load_dotenv
load_dotenv()

os.environ["HF_TOKEN"] = os.getenv("HF_TOKEN")

In [7]:
from langchain_huggingface import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

In [14]:
from langchain_community.vectorstores import Chroma

vectorStore = Chroma.from_documents(documents, embedding=embeddings)

query = "What are docker containers"
results = vectorStore.similarity_search(query, k=3)
for i, res in enumerate(results):
    print(f"Result {i+1}: {res.page_content}\n")

Result 1: Docker containers help maintain consistent development environments.

Result 2: Docker containers help maintain consistent development environments.

Result 3: Docker containers help maintain consistent development environments.



In [15]:
score = vectorStore.similarity_search_with_score(query)
score

[(Document(metadata={'source': 'https://docker.com'}, page_content='Docker containers help maintain consistent development environments.'),
  0.4634406566619873),
 (Document(metadata={'source': 'https://docker.com'}, page_content='Docker containers help maintain consistent development environments.'),
  0.4634406566619873),
 (Document(metadata={'source': 'https://docker.com'}, page_content='Docker containers help maintain consistent development environments.'),
  0.4634406566619873),
 (Document(metadata={'source': 'https://nodejs.org'}, page_content='Node.js is widely used for building scalable backend services.'),
  1.503739833831787)]

### Retrievers

In [10]:
# method 1(not suggested)

from langchain_core.runnables import RunnableLambda

retriever = RunnableLambda(vectorStore.similarity_search).bind(k=1)
retriever.batch(["docker", "Node.js "])

[[Document(metadata={'source': 'https://docker.com'}, page_content='Docker containers help maintain consistent development environments.')],
 [Document(metadata={'source': 'https://nodejs.org'}, page_content='Node.js is widely used for building scalable backend services.')]]

In [16]:
# method 2 (suggested)

retriever = vectorStore.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 1}
)
retriever.batch(["docker", "Node.js "])

[[Document(metadata={'source': 'https://docker.com'}, page_content='Docker containers help maintain consistent development environments.')],
 [Document(metadata={'source': 'https://nodejs.org'}, page_content='Node.js is widely used for building scalable backend services.')]]

In [18]:
from langchain_groq import ChatGroq

llm = ChatGroq(
    model="qwen/qwen3-32b",
    temperature=0,
    max_tokens=None,
    reasoning_format="parsed",
    timeout=None,
    max_retries=2,
    # other params...
)

In [22]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. "
            "Answer strictly using the provided context. "
            "If the answer is not in the context, say you don't know.",
        ),
        ("human", "Context:\n{context}\n\nQuestion:\n{question}"),
    ]
)

output_parser = StrOutputParser()

rag_chain = (
    {
        "context": retriever,
        "question": RunnablePassthrough(),
    }
    | prompt
    | llm
    | output_parser
)

response = rag_chain.invoke("tell me about docker")
print(response)

# Note : tell me about docker is the RunnablePassthrough()

Docker is a platform that uses containers to help maintain consistent development environments. Containers package applications along with their dependencies, ensuring they run reliably across different systems and stages of development (e.g., from a developer's laptop to production servers). This consistency reduces issues caused by differences in environments, such as "it works on my machine" problems. The information is sourced from Docker's official documentation at [docker.com](https://docker.com).
